In [ ]:
# Do not do run all, run the cells one at a time
import numpy as np

import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
from torch.utils.data import ConcatDataset
from skimage.feature import local_binary_pattern
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
IMG_HEIGHT = 48
IMG_WIDTH = 48

# Path to the training data
TRAIN_DATA_PATH = os.path.join(os.getcwd(), '../FER with DL/data', 'train')

# Path to the test data
TEST_DATA_PATH = os.path.join(os.getcwd(), '../FER with DL/data', 'test')

# Define your transformations
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor()
])

# Load the datasets
train_dataset = ImageFolder(TRAIN_DATA_PATH, transform=transform)
test_dataset = ImageFolder(TEST_DATA_PATH, transform=transform)

# Create the dataloader for validation set only, train data still needs to be augmented
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Takes roughly 4 minutes to run this cell

# Oversampling the disgust samples since we don't have many samples

# Define additional transformations for data augmentation
augment_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Create a new dataset with only the "disgust" images
disgust_dataset = [img for img in train_dataset if img[1]
                   == train_dataset.class_to_idx['disgust']]

In [ ]:
# Convert Tensor to PIL Image
to_pil = ToPILImage()

# Apply data augmentation to the "disgust" images
augmented_disgust_dataset = [(augment_transform(to_pil(img[0])), img[1])
                             for _ in range(3) for img in disgust_dataset]

In [ ]:
print(len(augmented_disgust_dataset))

In [ ]:
# Combine the original dataset with the augmented "disgust" images
train_dataset = ConcatDataset([train_dataset, augmented_disgust_dataset])

# Update the train DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Print details about the train and test datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

In [ ]:
# Local Binary patterns (LBP) Implementation - 35% Accuracy
def extract_features_lbp(data_loader):
    # Parameters for LBP
    radius = 3
    n_points = 8 * radius
    # Initialize an empty list to store the feature vectors
    features = []
    labels = []
    # Process each batch of images
    for images, batch_labels in data_loader:
        # Process each image in the batch
        for i in range(len(images)):
            image = images[i].numpy().squeeze()
            # Apply LBP
            lbp = local_binary_pattern(image, n_points, radius)
            # Flatten the LBP image and add it to the list of feature vectors
            features.append(lbp.ravel())
            # Add the corresponding label to the labels list
            labels.append(batch_labels[i].item())
    return features, labels

In [ ]:
# Eigenfaces Implmentation - 14% Accuracy
def extract_features_eigenfaces(data_loader):
    # Initialize an empty list to store the images and labels
    images = []
    labels = []
    # Process each batch of images
    for batch_images, batch_labels in data_loader:
        # Process each image in the batch
        for i in range(len(batch_images)):
            image = batch_images[i].numpy().squeeze()
            # Flatten the image and add it to the list of images
            images.append(image.ravel())
            # Add the corresponding label to the labels list
            labels.append(batch_labels[i].item())
    # Convert the lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    # Apply PCA to the images
    pca = PCA(n_components=150)
    features = pca.fit_transform(images)
    return features, labels

In [ ]:
# Fisherfaces Implementation - 12.45% Accuracy
def extract_features_fisherfaces(data_loader):
    # Initialize an empty list to store the images and labels
    images = []
    labels = []
    # Process each batch of images
    for batch_images, batch_labels in data_loader:
        # Process each image in the batch
        for i in range(len(batch_images)):
            image = batch_images[i].numpy().squeeze()
            # Flatten the image and add it to the list of images
            images.append(image.ravel())
            # Add the corresponding label to the labels list
            labels.append(batch_labels[i].item())
    # Convert the lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    # Apply PCA to the images
    pca = PCA(n_components=150)
    pca_result = pca.fit_transform(images)
    # Apply LDA to the PCA result
    lda = LDA(n_components=None)  # Automatically set n_components to min(n_features, n_classes - 1)
    features = lda.fit_transform(pca_result, labels)
    return features, labels

In [ ]:
# Takes roughly 4 minutes to run this cell
X_train, y_train = extract_features_eigenfaces(train_loader)

In [ ]:
# Takes roughly 4 minutes to run this cell
X_test, y_test = extract_features_eigenfaces(test_loader)

In [ ]:

print(f"Number of feature vectors: {len(X_train)}")
print(f"Shape of a feature vector: {X_train[0].shape}")


In [ ]:
def train_and_evaluate(X_train, y_train, X_test, y_test):
    # Create a SVM (Support Vector Machine) classifier
    clf = svm.SVC()

    # Train the classifier with the training data and labels
    clf.fit(X_train, y_train)

    # Predict the labels of the test data
    y_pred = clf.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate precision, recall, and F1-score
    report = classification_report(y_test, y_pred)

    print(f"Accuracy: {accuracy * 100}%")
    print("Classification Report:")
    print(report)

In [ ]:
train_and_evaluate(X_train, y_train, X_test, y_test)

In [ ]:
print(test_dataset.class_to_idx)